In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Exploratory Data Analysis (EDA)\n",
    "This notebook performs comprehensive exploratory data analysis on the cleaned churn dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "\n",
    "# Import custom modules\n",
    "from src.visualization import (\n",
    "    plot_churn_distribution, plot_feature_distribution,\n",
    "    plot_correlation_heatmap, create_interactive_churn_analysis\n",
    ")\n",
    "\n",
    "# Settings\n",
    "pd.set_option('display.max_columns', None)\n",
    "plt.style.use('seaborn-darkgrid')\n",
    "%matplotlib inline\n",
    "\n",
    "# Load cleaned data\n",
    "df = pd.read_csv('../1_data/processed/churn_data_features.csv')\n",
    "print(f\"Data loaded: {df.shape}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Univariate Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Churn distribution\n",
    "plot_churn_distribution(df)\n",
    "\n",
    "# Numerical features distribution\n",
    "numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges', \n",
    "                     'avg_monthly_charge', 'services_count']\n",
    "\n",
    "fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n",
    "axes = axes.ravel()\n",
    "\n",
    "for idx, feature in enumerate(numerical_features):\n",
    "    df[feature].hist(bins=30, ax=axes[idx], color='skyblue', edgecolor='black')\n",
    "    axes[idx].set_title(f'Distribution of {feature}')\n",
    "    axes[idx].set_xlabel(feature)\n",
    "    axes[idx].set_ylabel('Frequency')\n",
    "\n",
    "# Remove extra subplot\n",
    "fig.delaxes(axes[5])\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Summary statistics\n",
    "print(\"\\nNumerical Features Summary:\")\n",
    "print(df[numerical_features].describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Bivariate Analysis - Churn vs Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Categorical features vs Churn\n",
    "categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',\n",
    "                       'Contract', 'PaperlessBilling', 'PaymentMethod',\n",
    "                       'InternetService', 'tenure_group']\n",
    "\n",
    "fig, axes = plt.subplots(3, 3, figsize=(18, 15))\n",
    "axes = axes.ravel()\n",
    "\n",
    "for idx, feature in enumerate(categorical_features):\n",
    "    # Calculate churn rate by category\n",
    "    churn_by_cat = df.groupby(feature)['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)\n",
    "    \n",
    "    # Plot\n",
    "    churn_by_cat.plot(kind='bar', ax=axes[idx], color='coral')\n",
    "    axes[idx].set_title(f'Churn Rate by {feature}')\n",
    "    axes[idx].set_xlabel(feature)\n",
    "    axes[idx].set_ylabel('Churn Rate (%)')\n",
    "    axes[idx].set_ylim(0, 60)\n",
    "    \n",
    "    # Add value labels\n",
    "    for i, v in enumerate(churn_by_cat):\n",
    "        axes[idx].text(i, v + 1, f'{v:.1f}%', ha='center')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Numerical features vs Churn (Box plots)\n",
    "fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n",
    "axes = axes.ravel()\n",
    "\n",
    "for idx, feature in enumerate(numerical_features):\n",
    "    df.boxplot(column=feature, by='Churn', ax=axes[idx])\n",
    "    axes[idx].set_title(f'{feature} by Churn Status')\n",
    "    axes[idx].set_xlabel('Churn')\n",
    "    axes[idx].set_ylabel(feature)\n",
    "\n",
    "# Remove extra subplot\n",
    "fig.delaxes(axes[5])\n",
    "plt.suptitle('')  # Remove default title\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Customer Segment Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Churn rate by customer segments\n",
    "segments = [\n",
    "    ('Contract Type', 'Contract'),\n",
    "    ('Internet Service', 'InternetService'),\n",
    "    ('Payment Method', 'PaymentMethod'),\n",
    "    ('Tenure Group', 'tenure_group')\n",
    "]\n",
    "\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n",
    "axes = axes.ravel()\n",
    "\n",
    "for idx, (title, feature) in enumerate(segments):\n",
    "    # Create crosstab\n",
    "    ct = pd.crosstab(df[feature], df['Churn'], normalize='index') * 100\n",
    "    \n",
    "    # Plot stacked bar\n",
    "    ct.plot(kind='bar', stacked=True, ax=axes[idx], \n",
    "            color=['#2ecc71', '#e74c3c'])\n",
    "    axes[idx].set_title(f'Churn Distribution by {title}')\n",
    "    axes[idx].set_xlabel(feature)\n",
    "    axes[idx].set_ylabel('Percentage')\n",
    "    axes[idx].legend(['No Churn', 'Churn'])\n",
    "    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prepare data for correlation\n",
    "df_corr = df.copy()\n",
    "df_corr['Churn_Binary'] = (df_corr['Churn'] == 'Yes').astype(int)\n",
    "\n",
    "# Encode categorical variables for correlation\n",
    "for col in ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']:\n",
    "    df_corr[col + '_Binary'] = (df_corr[col] == 'Yes').astype(int)\n",
    "\n",
    "# Select features for correlation\n",
    "corr_features = ['tenure', 'MonthlyCharges', 'TotalCharges', 'services_count',\n",
    "                 'has_streaming', 'high_risk_payment', 'month_to_month',\n",
    "                 'no_online_services', 'Churn_Binary'] + \\\n",
    "                [col + '_Binary' for col in ['gender', 'Partner', 'Dependents', \n",
    "                                            'PhoneService', 'PaperlessBilling']]\n",
    "\n",
    "# Calculate correlation matrix\n",
    "corr_matrix = df_corr[corr_features].corr()\n",
    "\n",
    "# Plot correlation heatmap\n",
    "plt.figure(figsize=(12, 10))\n",
    "mask = np.triu(np.ones_like(corr_matrix, dtype=bool))\n",
    "sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', \n",
    "            cmap='coolwarm', center=0, vmin=-1, vmax=1,\n",
    "            square=True, linewidths=.5, cbar_kws={\"shrink\": .8})\n",
    "plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Top correlations with churn\n",
    "churn_correlations = corr_matrix['Churn_Binary'].sort_values(ascending=False)\n",
    "print(\"\\nTop Features Correlated with Churn:\")\n",
    "print(churn_correlations[1:11])  # Exclude self-correlation"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Interactive Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create interactive visualizations\n",
    "fig1, fig2, fig3 = create_interactive_churn_analysis(df)\n",
    "\n",
    "# Display interactive plots\n",
    "fig1.show()\n",
    "fig2.show()\n",
    "fig3.show()\n",
    "\n",
    "# Churn rate by multiple dimensions\n",
    "fig = px.sunburst(df, path=['Contract', 'InternetService', 'Churn'], \n",
    "                  title='Customer Distribution by Contract, Internet Service, and Churn')\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Key Insights Summary\n",
    "\n",
    "### High Risk Factors for Churn:\n",
    "1. **Contract Type**: Month-to-month contracts have 42% churn rate vs 11% for yearly contracts\n",
    "2. **Tenure**: New customers (0-12 months) have highest churn rate at 48%\n",
    "3. **Payment Method**: Electronic check users have 45% churn rate\n",
    "4. **Internet Service**: Fiber optic customers show higher churn (30%) than DSL (20%)\n",
    "5. **Senior Citizens**: 42% churn rate vs 24% for non-seniors\n",
    "\n",
    "### Protective Factors:\n",
    "1. **Long tenure**: Customers >48 months have only 15% churn rate\n",
    "2. **Multiple services**: Customers with 4+ services show lower churn\n",
    "3. **Having dependents**: 15% churn rate vs 32% without dependents\n",
    "4. **Two-year contracts**: Lowest churn rate at 3%\n",
    "\n",
    "### Business Recommendations:\n",
    "1. Target month-to-month customers for contract upgrades\n",
    "2. Implement retention programs for customers in first year\n",
    "3. Encourage electronic check users to switch payment methods\n",
    "4. Bundle services to increase customer stickiness"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}